In [1]:
import pandas as pd
import pickle
from itertools import compress
import numpy as np

# Database of awarded public procurement

In [2]:
data_old = pickle.load(open("../../data/interim/adjudicaciones_old.pickle",'rb'))
data_new = pickle.load(open("../../data/interim/adjudicaciones.pickle",'rb'))

In [3]:
colnames_old = pd.DataFrame([*map(lambda y: y.columns, data_old)])
colnames_old = colnames_old.apply(lambda y: (y.value_counts().idxmax(), y.value_counts().max(), y.unique())).transpose()
colnames = pd.DataFrame([*map(lambda y: y.columns, data_new)])
colnames = colnames.apply(lambda y: (y.value_counts().idxmax(), y.value_counts().max(), y.unique())).transpose()
colnames_old.to_excel("../../data/external/colnames_old_.xlsx")
colnames.to_excel("../../data/external/colnames_.xlsx")

In [4]:
rename_df = pd.read_excel("../../data/external/columns_names_adjudicaciones.xlsx", sheet_name='rename')
rename_df = rename_df.set_index('ORIGINAL')['MODIFICADO']

In [5]:
try:
    data_old = pd.concat(map(lambda df: df.rename(columns = rename_df), data_old))
except:
    data_old = pd.DataFrame()

In [6]:
data_new = pd.concat(map(lambda df: df, data_new))
data_adj = pd.concat([data_old, data_new], ignore_index=True)

In [8]:
data_adj['FECHACONVOCATORIA'] = pd.to_datetime(data_adj['FECHACONVOCATORIA'])
data_adj['MONTO_REFERENCIAL_ITEM'] = pd.to_numeric(data_adj['MONTO_REFERENCIAL_ITEM'])
data_adj['MONTO_REFERENCIAL_ITEM_SOLES'] = pd.to_numeric(data_adj['MONTO_REFERENCIAL_ITEM_SOLES'])
data_adj['MONTO_ADJUDICADO_ITEM'] = pd.to_numeric(data_adj['MONTO_ADJUDICADO_ITEM'])
data_adj['MONTO_ADJUDICADO_ITEM_SOLES'] = pd.to_numeric(data_adj['MONTO_ADJUDICADO_ITEM_SOLES'])
data_adj['CANTIDAD_ADJUDICADA_ITEM'] = pd.to_numeric(data_adj['CANTIDAD_ADJUDICADA_ITEM'])
data_adj['AÑO'] = data_adj['ORIGEN'].apply(lambda x: str(x)[-11:-7])

In [10]:
data_adj = data_adj[['AÑO','VERSION','CODIGOCONVOCATORIA','RUC_ENTIDAD','ENTIDAD','TIPOENTIDAD','TIPOPROCESOSELECCION','PROCESO','REGIMEN','N_ITEM',
             'DESCRIPCION_ITEM','UNIDAD_MEDIDA','PAQUETE','CODIGOGRUPO','DESCRIPCION_GRUPO','ITEMCUBSO','MONEDA',
             'MONTO_REFERENCIAL_ITEM','CANTIDAD_ADJUDICADA_ITEM','MONTO_ADJUDICADO_ITEM','RUC_CODIGO_GANADOR',
             'GANADOR','TIPO_PROVEEDOR','CONSORCIO', 'RUC_MIEMBRO','MIEMBRO','ORIGEN']]

In [11]:
orden_columnas = ['AÑO','VERSION','CODIGOCONVOCATORIA','N_ITEM','RUC_ENTIDAD','ENTIDAD','TIPOENTIDAD','TIPOPROCESOSELECCION','PROCESO','REGIMEN',
                  'DESCRIPCION_ITEM','UNIDAD_MEDIDA','PAQUETE','CODIGOGRUPO','DESCRIPCION_GRUPO','ITEMCUBSO','MONEDA',
                  'MONTO_REFERENCIAL_ITEM','CANTIDAD_ADJUDICADA_ITEM','MONTO_ADJUDICADO_ITEM','RUC_CODIGO_GANADOR',
                  'GANADOR','TIPO_PROVEEDOR','CONSORCIO', 'RUC_MIEMBRO','MIEMBRO','ORIGEN']
data_adj = data_adj.reindex(columns=orden_columnas)

# Bidders' database

In [12]:
data_old_postores = pickle.load(open("../../data/interim/postores_old.pickle",'rb'))
data_new_postores = pickle.load(open("../../data/interim/postores.pickle",'rb'))

In [13]:
rename_df = pd.read_excel("../../data/external/columns_names_postores.xlsx", sheet_name='rename')
rename_df2 = pd.read_excel("../../data/external/columns_names_postores.xlsx", sheet_name='tonew')
rename_df2 = rename_df2[rename_df2.ORIGINAL != rename_df2.MODIFICADO]

rename_df = rename_df.set_index('ORIGINAL')['MODIFICADO']
rename_df2 = rename_df2.set_index('ORIGINAL')['MODIFICADO']

In [14]:
try:
    data_old_postores = pd.concat(map(lambda df: df.rename(columns = rename_df).rename(columns = rename_df2), data_old_postores))
except:
    data_old_postores = pd.DataFrame()

In [16]:
data_new_postores = pd.concat(map(lambda df: df.rename(columns = rename_df).rename(columns = rename_df2), data_new_postores))
data_postores = pd.concat([data_old_postores, data_new_postores], ignore_index=True)

In [18]:
data_postores = data_postores[['CODIGO_CONVOCATORIA','FECHACONVOCATORIA','FECHA_REGISTRO_SISTEMA',
                               'HORA_REGISTRO','FECHA_PRESENTACION_PROPUESTA', 'HORA_PRESENTACION',
                               'N_ITEM','RUC_CODIGO_POSTOR','POSTOR','RUC_DETALLEPOSTOR','DETALLE_POSTOR', 'PORCENTAJE',
                               'MONEDA','MONTO','ESTADOADMISIONPRESENPROPUESTA', 'ORIGEN']]

In [19]:
data_postores.columns = ['CODIGOCONVOCATORIA','FECHACONVOCATORIA','FECHA_REGISTRO_SISTEMA',
                         'HORA_REGISTRO','FECHA_PRESENTACION_PROPUESTA', 'HORA_PRESENTACION',
                         'N_ITEM','RUC_CODIGO_POSTOR','POSTOR','RUC_DETALLEPOSTOR','DETALLE_POSTOR', 'PORCENTAJE',
                         'MONEDA','MONTO','ESTADOADMISIONPRESENPROPUESTA', 'ORIGEN']

In [20]:
data_postores['MONTO'] = pd.to_numeric(data_postores['MONTO'].apply(str).str.replace(",","."), errors = 'coerce')
data_postores['PORCENTAJE'] = pd.to_numeric(data_postores['PORCENTAJE'])

In [21]:
data_postores = data_postores.drop_duplicates()

In [23]:
data_postores['MONTOF'] = data_postores.groupby(['CODIGOCONVOCATORIA','N_ITEM','RUC_CODIGO_POSTOR'])['MONTO'].transform('sum')

In [24]:
data_postores=data_postores.reset_index()

In [25]:
postores_unicos = data_postores.groupby(['CODIGOCONVOCATORIA','N_ITEM'])['RUC_CODIGO_POSTOR'].nunique()
postores_unicos=postores_unicos.reset_index()
postores_unicos.columns = ['CODIGOCONVOCATORIA','N_ITEM','NPOSTORES']
postores_unicos
data_postores = pd.merge(data_postores, postores_unicos, on=['CODIGOCONVOCATORIA','N_ITEM'],how='left')
data_postores=data_postores.reset_index()

# Geograpich database

In [28]:
data_old = pickle.load(open("../../data/interim/seace_old.pickle",'rb'))
data_new = pickle.load(open("../../data/interim/seace.pickle",'rb'))

colnames_old = pd.DataFrame([*map(lambda y: y.columns, data_old)])
colnames_old = colnames_old.apply(lambda y: (y.value_counts().idxmax(), y.value_counts().max(), y.unique())).transpose()
colnames = pd.DataFrame([*map(lambda y: y.columns, data_new)])
colnames = colnames.apply(lambda y: (y.value_counts().idxmax(), y.value_counts().max(), y.unique())).transpose()
colnames_old.to_excel("../../data/external/colnames_old_.xlsx")
colnames.to_excel("../../data/external/colnames_.xlsx")

rename_df = pd.read_excel("../../data/external/columns_names.xlsx", sheet_name='rename')
rename_df = rename_df.set_index('ORIGINAL').to_dict()['MODIFICADO']

In [29]:
try:
    data_old = pd.concat(map(lambda df: df.rename(columns = rename_df), data_old))
except:
    data_old = pd.DataFrame()

In [30]:
data_new = pd.concat(map(lambda df: df.rename(columns = rename_df), data_new))
data = pd.concat([data_old, data_new], ignore_index = True)

In [32]:
geodata = data[['CODIGOCONVOCATORIA', 'N_ITEM', 'RUC_ENTIDAD','ENTIDAD_DEPARTAMENTO',
             'ENTIDAD_DISTRITO', 'ENTIDAD_PROVINCIA']]

# Merge awarded public procurement, Bidders' database and geograpic database

In [34]:
df_combinado = pd.merge(data_adj, data_postores, on=['CODIGOCONVOCATORIA','N_ITEM'],how='left')
df_combinado_final = pd.merge(df_combinado, geodata, on=['CODIGOCONVOCATORIA','N_ITEM','RUC_ENTIDAD'],how='left')

# Filter the database of municipalities located in Lima

In [7]:
df_combinado_final= df_combinado_final[(df_combinado_final['ENTIDAD_DEPARTAMENTO'] == 'LIMA')]
df_combinado_final= df_combinado_final[(df_combinado_final['TIPOENTIDAD'] == 'GOBIERNO LOCAL')]
df_combinado_final

NameError: name 'df_combinado_final' is not defined